In [1]:
import findspark
findspark.init()
import pyspark
from random import random
from pyspark import SparkContext
import numpy as np
from math import sqrt

import matplotlib
import matplotlib.pyplot
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]

4197 mat614


In [3]:
sc = pyspark.SparkContext(appName="Netflix", master='',conf = config )

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

The data for this analysis can be obtained from [here](http://grouplens.org/datasets/movielens/).


Each line contains 4 columns:
1. user id 
2. movie id 
3. rating
4. timestamp 

In [4]:
trainData = sc.textFile('data/ml-100k/trainData')
testData = sc.textFile('data/ml-100k/testData')

print "We have ", trainData.count(), 'training raitings and',testData.count(),'test raitings'

We have  581140 training raitings and 118860 test raitings


In [5]:
def parseInput(s):
    data = s.split()
    return [int(data[0]),int(data[1]),int(data[2])]

top20Movies =  trainData.map(parseInput).map(lambda d:  (d[1],1)).reduceByKey(lambda x,y: x+y)\
           .takeOrdered(20,key = lambda x: -x[1])


In [6]:
print top20Movies

[(50, 3331), (181, 2921), (100, 2912), (258, 2857), (294, 2740), (286, 2706), (288, 2679), (1, 2605), (121, 2481), (174, 2443), (300, 2416), (127, 2377), (56, 2282), (98, 2265), (7, 2253), (237, 2205), (117, 2172), (172, 2126), (222, 2099), (204, 2033)]


In [7]:
movieInfo = open("data/ml-100k/u.item") 
dic = {}
for line in movieInfo:
    data = line.split("|")
    dic[ int(data[0]) ] =  data[1]
movieInfo.close()    
  
for e in top20Moviews:
    print e[1], e[0], dic[ e[0] ] 

3331 50 Star Wars (1977)
2921 181 Return of the Jedi (1983)
2912 100 Fargo (1996)
2857 258 Contact (1997)
2740 294 Liar Liar (1997)
2706 286 English Patient, The (1996)
2679 288 Scream (1996)
2605 1 Toy Story (1995)
2481 121 Independence Day (ID4) (1996)
2443 174 Raiders of the Lost Ark (1981)
2416 300 Air Force One (1997)
2377 127 Godfather, The (1972)
2282 56 Pulp Fiction (1994)
2265 98 Silence of the Lambs, The (1991)
2253 7 Twelve Monkeys (1995)
2205 237 Jerry Maguire (1996)
2172 117 Rock, The (1996)
2126 172 Empire Strikes Back, The (1980)
2099 222 Star Trek: First Contact (1996)
2033 204 Back to the Future (1985)


Lets create generic ratings for a cold user that is not in the provided dataset to make predictions for

In [51]:
myRatings = {} 
myRatings[300] = 5
myRatings[174] = 4
myRatings[204] = 5 
myRatings[121] = 4 
myRatings[1] = 3  
myRatings[56] = 3  

print myRatings

{1: 3, 300: 5, 204: 5, 174: 4, 56: 3, 121: 4}


# Train ALS model and compute loss (RMSE)

In [10]:
 def createRatings(d):
    return Rating( d[0], d[1], float(d[2]))

ratings = trainData.map(parseInput).map(createRatings) .cache()

rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

In [11]:
testDataToPredict =  testData.map(parseInput).map(lambda d: (d[0],d[1])) 
predictions = model.predictAll(testDataToPredict)

In [12]:
r = Rating( 17, 100, 4.5)
print r
print r.user
print r.rating
print r.product

Rating(user=17, product=100, rating=4.5)
17
4.5
100


In [13]:
print predictions.take(10)
predictionsForJoin = predictions.map(lambda r: ( (r.user,r.product)  , r.rating ) )
print predictionsForJoin.take(10)


trueTest =  testData.map(parseInput) 

print "\n\n"

print trueTest.take(10)
trueTestForJoin = trueTest.map(lambda d: ( (d[0],d[1])  , d[2] ) )
print trueTestForJoin.take(10)


print "\n\n"

jointedData = trueTestForJoin.join(predictionsForJoin)
print jointedData.take(10) 

[Rating(user=315, product=1084, rating=4.092592143136823), Rating(user=195, product=1084, rating=3.4980653165827587), Rating(user=195, product=1084, rating=3.4980653165827587), Rating(user=541, product=1084, rating=4.505339296423282), Rating(user=181, product=1084, rating=1.2560875772961828), Rating(user=601, product=1084, rating=4.842021855372476), Rating(user=821, product=1084, rating=4.740700930738167), Rating(user=821, product=1084, rating=4.740700930738167), Rating(user=655, product=1084, rating=2.9457359998731723), Rating(user=807, product=1084, rating=4.268434463916589)]
[((315, 1084), 4.092592143136823), ((195, 1084), 3.4980653165827587), ((195, 1084), 3.4980653165827587), ((541, 1084), 4.505339296423282), ((181, 1084), 1.2560875772961828), ((601, 1084), 4.842021855372476), ((821, 1084), 4.740700930738167), ((821, 1084), 4.740700930738167), ((655, 1084), 2.9457359998731723), ((807, 1084), 4.268434463916589)]



[[1, 6, 5], [1, 10, 3], [1, 12, 5], [1, 14, 5], [1, 17, 3], [1, 20,

In [14]:
print jointedData.take(10)

def computeError(d):
    k = d[0]
    v = d[1]
    error = v[1]-v[0]
    return error**2

print jointedData.map(computeError).take(10)
print jointedData.map(computeError).mean()


[((506, 568), (5, 4.274867044539546)), ((109, 365), (4, 4.633545863891776)), ((711, 707), (5, 4.983291481955373)), ((711, 707), (5, 4.983291481955373)), ((711, 707), (5, 4.983291481955373)), ((711, 707), (5, 4.983291481955373)), ((720, 286), (5, 4.146478850305002)), ((720, 286), (5, 4.146478850305002)), ((720, 286), (5, 4.146478850305002)), ((720, 286), (5, 4.146478850305002))]
[0.5258178030948131, 0.40138036165437635, 0.00027917457524763076, 0.00027917457524763076, 0.00027917457524763076, 0.00027917457524763076, 0.7284983529766719, 0.7284983529766719, 0.7284983529766719, 0.7284983529766719]
0.469572866407


# Get average ratings overall, by movie, and by user

In [16]:
mu = 0 # this should be an overall average rating
userBias = {}  # this should be a hash-table containing user averages
movieBias = {}  # this should be a hash-table containing movie averages


ratings = trainData.map(parseInput).map(createRatings).cache()
print ratings.take(10)

[Rating(user=1, product=1, rating=5.0), Rating(user=1, product=2, rating=3.0), Rating(user=1, product=3, rating=4.0), Rating(user=1, product=4, rating=3.0), Rating(user=1, product=5, rating=3.0), Rating(user=1, product=7, rating=4.0), Rating(user=1, product=8, rating=1.0), Rating(user=1, product=9, rating=5.0), Rating(user=1, product=11, rating=2.0), Rating(user=1, product=13, rating=5.0)]


In [17]:
print ratings.take(10)
mu = ratings.map(lambda r: r.rating).mean()
print "My average rating is:",mu

[Rating(user=1, product=1, rating=5.0), Rating(user=1, product=2, rating=3.0), Rating(user=1, product=3, rating=4.0), Rating(user=1, product=4, rating=3.0), Rating(user=1, product=5, rating=3.0), Rating(user=1, product=7, rating=4.0), Rating(user=1, product=8, rating=1.0), Rating(user=1, product=9, rating=5.0), Rating(user=1, product=11, rating=2.0), Rating(user=1, product=13, rating=5.0)]
My average rating is: 3.52795367726


In [18]:
print ratings.take(10)
print ratings.map(lambda r: (r.user, r.rating)).take(10)
print "\n\n"
print ratings.map(lambda r: (r.user, np.array([r.rating,1]))).take(10)
print "\n\n"
print ratings.map(lambda r: (r.user, np.array([r.rating,1]))).reduceByKey(lambda x,y: x+y).take(10)
print "\n\n"
userRating =  ratings.map(lambda r: (r.user, np.array([r.rating,1]))).reduceByKey(lambda x,y: x+y)\
           .map(lambda d:  (d[0], d[1][0]/d[1][1]  ) ).collect()

[Rating(user=1, product=1, rating=5.0), Rating(user=1, product=2, rating=3.0), Rating(user=1, product=3, rating=4.0), Rating(user=1, product=4, rating=3.0), Rating(user=1, product=5, rating=3.0), Rating(user=1, product=7, rating=4.0), Rating(user=1, product=8, rating=1.0), Rating(user=1, product=9, rating=5.0), Rating(user=1, product=11, rating=2.0), Rating(user=1, product=13, rating=5.0)]
[(1, 5.0), (1, 3.0), (1, 4.0), (1, 3.0), (1, 3.0), (1, 4.0), (1, 1.0), (1, 5.0), (1, 2.0), (1, 5.0)]



[(1, array([5., 1.])), (1, array([3., 1.])), (1, array([4., 1.])), (1, array([3., 1.])), (1, array([3., 1.])), (1, array([4., 1.])), (1, array([1., 1.])), (1, array([5., 1.])), (1, array([2., 1.])), (1, array([5., 1.]))]



[(2, array([1308.,  352.])), (4, array([537., 124.])), (6, array([4529., 1246.])), (8, array([1267.,  334.])), (10, array([4560., 1084.])), (12, array([1251.,  286.])), (14, array([2324.,  568.])), (16, array([3551.,  820.])), (18, array([6376., 1642.])), (20, array([839., 268.]

In [19]:
for e in  userRating:
    userBias[e[0]]= e[1]-mu

In [20]:
movieRating =  ratings.map(lambda r: (r.product, np.array([r.rating,1]))).reduceByKey(lambda x,y: x+y)\
           .map(lambda d:  (d[0], d[1][0]/d[1][1]  ) ).collect()

In [22]:
for e in  movieRating:
    movieBias[e[0]]= e[1]-mu
print movieBias    

{1: 0.346902368810202, 2: -0.32146017076155076, 3: -0.49183200425125495, 4: 0.02082681054982105, 5: -0.2289437762649582, 6: 0.026100376798996905, 7: 0.2709810764067271, 8: 0.46890660531951145, 9: 0.36602083027680843, 10: 0.297289041191545, 11: 0.3192078947973447, 12: 0.8581912874659179, 13: -0.10941435141236022, 14: 0.43299870369732396, 15: 0.2488959647497162, 16: -0.3244904737918537, 17: -0.40341836126992714, 18: -0.7279536772550572, 19: 0.413965514664135, 20: -0.12072476159240653, 21: -0.7682000838258989, 22: 0.6258481660629154, 23: 0.590134420214484, 24: -0.0832896456345038, 25: -0.09396907097915408, 26: -0.08169199501206625, 27: -0.42049099068789264, 28: 0.40612592473499287, 29: -0.8622688466217729, 30: 0.41623236925657103, 31: 0.10216813891327092, 32: 0.2547467446858711, 33: -0.07372832514238103, 34: -0.8694170918892032, 35: -1.3461354954368754, 36: -1.407953677255057, 37: -1.293911124063568, 38: -0.5150965343979141, 39: -0.2622056457589941, 40: -0.6410489153502952, 41: -0.4457618

In [23]:
print myRatings

{1: 1, 300: 5, 204: 5, 174: 5, 56: 1, 121: 5}


In [52]:
myRatingsData = []
for key in myRatings:
    myRatingsData.append( [0, key, myRatings[key] ] )
myRatingsDataRDD = sc.parallelize(myRatingsData)    
print myRatingsDataRDD.collect()

[[0, 1, 3], [0, 300, 5], [0, 204, 5], [0, 174, 4], [0, 56, 3], [0, 121, 4]]


In [53]:
print myRatingsDataRDD.count()
print trainData.count()
ALL_DATA = myRatingsDataRDD.union( trainData.map(lambda l: [int(x) for x in l.split()[0:3]] )  )
print ALL_DATA.count()
print ALL_DATA.take(10)


6
581140
581146
[[0, 1, 3], [0, 300, 5], [0, 204, 5], [0, 174, 4], [0, 56, 3], [0, 121, 4], [1, 1, 5], [1, 2, 3], [1, 3, 4], [1, 4, 3]]


In [55]:
userBias[0] = sum(myRatings.values())/6.0 - mu
print userBias[0]

0.472046322745


In [56]:
# Write the solution here
finalTrainData = ALL_DATA .map(createRatings).cache()
print finalTrainData.take(10)

[Rating(user=0, product=1, rating=3.0), Rating(user=0, product=300, rating=5.0), Rating(user=0, product=204, rating=5.0), Rating(user=0, product=174, rating=4.0), Rating(user=0, product=56, rating=3.0), Rating(user=0, product=121, rating=4.0), Rating(user=1, product=1, rating=5.0), Rating(user=1, product=2, rating=3.0), Rating(user=1, product=3, rating=4.0), Rating(user=1, product=4, rating=3.0)]


In [57]:
def removeBaseline(r):  # this removes the baseline rating from the training data
    baselineEstimate = mu+ userBias[r.user]  +  movieBias[r.product]
    
    return Rating(r.user,r.product,r.rating - baselineEstimate)
print finalTrainData.take(10)    
print "\n"
print mu, userBias[0], movieBias[1]   
print "\n"
print finalTrainData.map(removeBaseline) .take(10)   

[Rating(user=0, product=1, rating=3.0), Rating(user=0, product=300, rating=5.0), Rating(user=0, product=204, rating=5.0), Rating(user=0, product=174, rating=4.0), Rating(user=0, product=56, rating=3.0), Rating(user=0, product=121, rating=4.0), Rating(user=1, product=1, rating=5.0), Rating(user=1, product=2, rating=3.0), Rating(user=1, product=3, rating=4.0), Rating(user=1, product=4, rating=3.0)]


3.52795367726 0.472046322745 0.346902368810202


[Rating(user=0, product=1, rating=-1.346902368810202), Rating(user=0, product=300, rating=0.9004702335464483), Rating(user=0, product=204, rating=0.693718556743498), Rating(user=0, product=174, rating=-0.7254233182422816), Rating(user=0, product=56, rating=-1.5404074095109372), Rating(user=0, product=121, rating=0.09707902993542783), Rating(user=1, product=1, rating=1.0476385741178378), Rating(user=1, product=2, rating=-0.2839988863104095), Rating(user=1, product=3, rating=0.8863729471792947), Rating(user=1, product=4, rating=-0.62628586762178

In [59]:
rank = 2
numIterations = 10
ratingsWithRemovedBaseline = finalTrainData.map(removeBaseline)
model = ALS.train(ratingsWithRemovedBaseline, rank, numIterations)

# What movies should I watch on Friday?

In [60]:
allMoviesRDD = sc.parallelize(movieBias.keys())
print allMoviesRDD.take(10)

predictForMe = allMoviesRDD.map(lambda d: (0,d))
print predictForMe.take(10)

predForMe = model.predictAll(predictForMe)
print "\n\n"
print predForMe.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10)]



[Rating(user=0, product=1084, rating=-0.22260317089763415), Rating(user=0, product=454, rating=0.6874995809202566), Rating(user=0, product=1410, rating=0.8964931210098968), Rating(user=0, product=772, rating=-2.166031441011244), Rating(user=0, product=752, rating=-0.2929303007018993), Rating(user=0, product=586, rating=0.26180356034410224), Rating(user=0, product=428, rating=-0.9805347110355473), Rating(user=0, product=1328, rating=-0.9295525585258213), Rating(user=0, product=464, rating=0.09811809102738445), Rating(user=0, product=14, rating=-0.731823989262903)]


In [61]:
def addBaselineEstimate(r):
    baselineEstimate = mu+ userBias[r.user]  +  movieBias[r.product]
    
    return Rating(r.user,r.product,r.rating + baselineEstimate)
 
print predForMe.map(addBaselineEstimate).take(10)

[Rating(user=0, product=1084, rating=4.092418358458879), Rating(user=0, product=454, rating=4.027677771797068), Rating(user=0, product=1410, rating=4.118539443754839), Rating(user=0, product=772, rating=1.3611872955268027), Rating(user=0, product=752, rating=3.48542232834935), Rating(user=0, product=586, rating=3.3988498830890452), Rating(user=0, product=428, rating=3.303309364518384), Rating(user=0, product=1328, rating=2.6853509070762644), Rating(user=0, product=464, rating=4.160226525573569), Rating(user=0, product=14, rating=3.701174714434421)]


In [62]:
watchOnFriday = predForMe.map(addBaselineEstimate).takeOrdered(10, lambda r: -r.rating)

In [63]:
for r in watchOnFriday:
    print r.product,dic[r.product], r.rating

1306 Delta of Venus (1994) 13.5223260638
1304 New York Cop (1996) 12.7459943388
851 Two or Three Things I Know About Her (1966) 8.5800048708
1594 Everest (1998) 8.54977453804
1104 C'est arriv� pr�s de chez vous (1992) 7.95859628564
1463 Boys, Les (1997) 7.88351556215
1360 Sexual Life of the Belgians, The (1994) 7.71857845461
1347 Ballad of Narayama, The (Narayama Bushiko) (1958) 7.14817405247
1260 Total Eclipse (1995) 7.14661297245
1631 Slingshot, The (1993) 7.11191619345


In [64]:
sc.stop()